<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/continuous_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ═══════════════════════════════════════════════════════════════
# FRAGILITY METRICS v9.6 — Continuous Outcomes Calculator
# Two modes: (1) Full data   (2) Only mean difference + 95% CI
# Fully compliant with FRAGILITY_METRICS.md v9.6 (22-Nov-2025)
# © Thomas F. Heston 2025 — CC-BY-4.0
# ═══════════════════════════════════════════════════════════════

import numpy as np
from scipy.stats import t
import ipywidgets as widgets
from IPython.display import display, Markdown

ALPHA = 0.05

# --------------------- MODE 1: Full data (m1,m2,sd1,sd2,n1,n2) ---------------------
def full_data_mode(m1, sd1, n1, m2, sd2, n2):
    se1 = sd1 / np.sqrt(n1)
    se2 = sd2 / np.sqrt(n2)
    se_diff = np.sqrt(se1**2 + se2**2)
    t_stat = (m1 - m2) / se_diff
    df = (se1**2 + se2**2)**2 / (se1**4/(n1-1) + se2**4/(n2-1))
    p_val = 2 * t.sf(np.abs(t_stat), df)
    t_crit = t.ppf(1 - ALPHA/2, df)
    cfs = abs(abs(t_stat) - t_crit)
    cfq = cfs / (1 + cfs)
    meci = abs(t_stat) / (1 + abs(t_stat))

    print("=== p–fr–nb (full data – exact v9.6) ===")
    print(f"Mean diff = {abs(m1-m2):.4f}   SE_diff = {se_diff:.4f}   |T| = {abs(t_stat):.3f}")
    print(f"p = {p_val:.6f}  →  {'significant' if p_val<=0.05 else 'not significant'}")
    print(f"fr: CFQ = {cfq:.3f}")
    print(f"nb: MeCI = {meci:.3f}")
    print("═" * 50)

# --------------------- MODE 2: Only mean difference + 95% CI ---------------------
# MODE 2: Only mean difference + 95% CI → now with CFS too
def ci_only_mode(delta, ci_lower, ci_upper):
    se_diff = (ci_upper - ci_lower) / 3.92          # 95% CI width ÷ (2×1.96)
    T_approx = abs(delta) / se_diff
    t_crit = 1.96                                   # conservative
    cfs = max(T_approx - t_crit, 0)                 # ← this is the official CFS
    cfq = cfs / (1 + cfs)
    meci = T_approx / (1 + T_approx)
    p_approx = 2 * (1 - t.cdf(T_approx, df=999))

    print("=== p–fr–nb (CI-only mode – v9.6) ===")
    print(f"Mean difference = {delta:.4f}")
    print(f"95% CI = [{ci_lower:.4f}, {ci_upper:.4f}] → SE_diff ≈ {se_diff:.4f}")
    print(f"Approximate |T| = {T_approx:.3f}")
    print()
    print(f"Secondary metric → CFS = {cfs:.3f} SE-units to p=0.05 boundary")
    print(f"fr: CFQ = {cfq:.3f}")
    print(f"nb: MeCI = {meci:.3f}")
    print(f"p ≈ {p_approx:.6f}  ({'significant' if p_approx<=0.05 else 'not significant'})")
    print("═" * 60)
#
# --------------------- Interactive widget ---------------------
print("Choose your input mode:\n")

mode = widgets.Dropdown(
    options=[('I have means + SDs + ns (exact)', 1),
             ('I only have mean difference + 95% CI (most papers)', 2)],
    value=2,
    description='Mode:'
)
display(mode)

# Containers for the two sets of inputs
full_inputs = widgets.VBox([
    widgets.FloatText(description='Group 1 mean'),
    widgets.FloatText(description='Group 1 SD'),
    widgets.IntText(description='Group 1 n'),
    widgets.FloatText(description='Group 2 mean'),
    widgets.FloatText(description='Group 2 SD'),
    widgets.IntText(description='Group 2 n')
])

ci_inputs = widgets.VBox([
    widgets.FloatText(description='Mean difference', value=1.19),
    widgets.FloatText(description='95% CI lower', value=0.13),
    widgets.FloatText(description='95% CI upper', value=2.25)
])

input_box = widgets.VBox([full_inputs])  # default
def update_inputs(change):
    if change['new'] == 1:
        input_box.children = [full_inputs]
    else:
        input_box.children = [ci_inputs]
mode.observe(update_inputs, names='value')

display(input_box)

button = widgets.Button(description="Calculate p–fr–nb", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        if mode.value == 1:
            m1  = full_inputs.children[0].value
            sd1 = full_inputs.children[1].value
            n1  = full_inputs.children[2].value
            m2  = full_inputs.children[3].value
            sd2 = full_inputs.children[4].value
            n2  = full_inputs.children[5].value
            full_data_mode(m1, sd1, n1, m2, sd2, n2)
        else:
            delta = ci_inputs.children[0].value
            low   = ci_inputs.children[1].value
            high  = ci_inputs.children[2].value
            ci_only_mode(delta, low, high)

button.on_click(on_button_clicked)
display(button, output)

Choose your input mode:



Dropdown(description='Mode:', index=1, options=(('I have means + SDs + ns (exact)', 1), ('I only have mean dif…

Button(button_style='success', description='Calculate p–fr–nb', style=ButtonStyle())

Output()